# Data analysis project

_In this project I am going to write a program to predict the number likes post is going to get in vk._

__Выполнил__: Булгаков Дмитрий (ИАД16)

__Дедлайн__: 23:59 10.04.16

# 1. Loading Data

VK library quiet installation and import into the notebook.

In [4]:
# !pip install vk # makes it quiet
import vk

Starting new vk session in order to parse data

In [5]:
vk_session = vk.Session() # starting new session
vk_api = vk.API(vk_session)

Getting number of posts in selected vk group.

In [6]:
selected_group = 'hse_overheard' # no other ideas :c
posts_number = vk_api.wall.get(domain=selected_group)[0] # number of posts is stored in first element
print('Number of posts in selected group: ', posts_number - 1)

Number of posts in selected group:  13964


Writing a function to parse more, than 100 posts from group.

In [7]:
def load_all_posts(page, n_posts, api):
    all_posts = api.wall.get(domain=page, count=n_posts)
    n_loaded = len(all_posts)
    while n_loaded < n_posts: # loop to load more, than 100 posts
        s = api.wall.get(domain=page, offset=n_loaded, count=(n_posts - n_loaded)) # update offset
        all_posts += s[1:] # no need for first element
        n_loaded += len(s) - 1 # update n_loaded
    return all_posts

Loading all posts from group for future analysis

In [45]:
try:
    loaded_posts = load_all_posts(page=selected_group, n_posts=1501, api=vk_api)[1:] # no need for posts number element
    # 1500 for this time, because I have small amount of ram avaliable :c
    print('Number of loaded posts: ', len(loaded_posts))
except: # timout errors are often to occur
    print('Error occured! Try again.')

Number of loaded posts:  1500


# 2. Data preprocessing

Loading required libs to preprocess data.

In [46]:
# !pip install pymorphy2 -q # silent install again
# !pip install stop_words -q # needed to remove stop words
from stop_words import get_stop_words
import pymorphy2 # need this one to convert words to normal time
import datetime # needed to convert response date 
import string # needed to work with strings
from nltk.tokenize import TweetTokenizer # needed to split text
import pandas as pd # required to work with dataframes
from ipywidgets import IntProgress # progressbar
from IPython.display import display # progressbar

Writing functions to process text data. Converting words to normal form and removing punctuation here.

In [47]:
def split_text(text):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(text) # spliting text into words

def convert_to_normal_form(words_list):
    morph = pymorphy2.MorphAnalyzer()
    normal_forms_list = []
    for word in words_list:
        if word not in string.punctuation and word[0] != "<":
            norm_form = morph.parse(word)[0].normal_form #getting normal form of a word
            normal_forms_list.append(norm_form) #adding it to list
    return normal_forms_list

def convert_text(text):
    words_list = split_text(text) # spliting text into words
    norm_words_list = convert_to_normal_form(words_list) # words into normal form
    filtered_words = [w for w in norm_words_list if not w in get_stop_words('russian')] # removing stop words
    return " ".join(filtered_words) # joining words to a sentence again

Writing a function to convert received list into another with another data.

In [48]:
def convert_posts(posts_list):
    progress = IntProgress() 
    progress.max = len(posts_list) # initializing progressbar
    progress.description = 'Processing data convertion'
    display(progress)
    
    updated_posts = [] # list of new posts' list structure
    for i, post in enumerate(posts_list): 
        tmp_dict = {} # creating empty dictionary for each post
        tmp_dict['likes_number'] = int(post['likes']['count']) # getting likes count
        tmp_dict['text'] = convert_text(post['text']) # converting text into normal form
        tmp_dict['text_length'] = len(post['text']) # calculating text length
        tmp_dict['post_hour'] = int(datetime.datetime.fromtimestamp(post['date']).strftime('%H')) # parsing only post hour
        tmp_dict['post_month'] = int(datetime.datetime.fromtimestamp(post['date']).strftime('%m')) # and post month
        tmp_dict['signed'] = int(post['from_id'] != -57354358) # checking whether post is signed or not
        # checking if any attacment exists
        tmp_dict['with_attachment'] = 1 if 'attachment' in post.keys() else 0
        tmp_dict['pinned'] = 1 if 'is_pinned' in post.keys() else 0 # cheking if post is pinned
        tmp_dict['repost'] = 1 if post['post_type'] == 'copy' else 0 # cheking if repost
        updated_posts.append(tmp_dict)
        progress.value += 1 # increasing progressbar value
    progress.description = 'Done convertion!'
    return updated_posts

Converting list of posts into new more convenient one.

In [49]:
converted_posts = convert_posts(loaded_posts)

# 3. Creating object-feature matrix

Loading pandas

In [50]:
import pandas as pd

Creating dataframe from parsed data

In [51]:
posts_frame = pd.DataFrame(converted_posts)
posts_frame.head()

,likes_number,pinned,post_hour,post_month,repost,signed,text,text_length,with_attachment
0,0,0,18,6,0,0,выпуск заниматься студент прикладной математик...,67,0
1,2,0,23,6,0,0,поступать юрфак набигай регион общага родитель...,426,0
2,4,0,14,6,0,0,рассказать чувствовать узнать поступить хороши...,110,0
3,29,0,9,6,0,0,удивительно 30-40 оставаться чайлдфри убедить ...,407,0
4,8,0,22,6,0,0,научный интерес 30 существовать феминистка арб...,412,0


And describing posts data

In [52]:
posts_frame.describe()

,likes_number,pinned,post_hour,post_month,repost,signed,text_length,with_attachment
count,1500.000000,1500.0,1500.000000,1500.000000,1500.000000,1500.0,1500.000000,1500.000000
mean,18.751333,0.0,15.180000,3.732000,0.029333,0.0,239.930667,0.223333
std,31.156197,0.0,6.067191,1.664339,0.168795,0.0,357.668800,0.416619
min,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.000000
25%,2.000000,0.0,12.000000,2.000000,0.000000,0.0,67.000000,0.000000
50%,7.000000,0.0,16.000000,4.000000,0.000000,0.0,134.000000,0.000000
75%,23.000000,0.0,20.000000,5.000000,0.000000,0.0,275.250000,0.000000
max,474.000000,0.0,23.000000,6.000000,1.000000,0.0,5035.000000,1.000000


Creating object-feature matrix

In [247]:
from sklearn.feature_extraction.text import TfidfVectorizer # loading count vectorizer

cv = TfidfVectorizer(norm='l1', max_features = 1000, analyzer = 'word', strip_accents='unicode', binary=True)
train_features = cv.fit_transform(posts_frame['text']).toarray() # vectorizing texts
train_frame = posts_frame.join(pd.DataFrame(train_features, columns=cv.get_feature_names())) # transfering it to pandas

train_frame.drop(['likes_number','text'],inplace=True,axis=1,errors='ignore') # removing unnecessary columns
value_frame = posts_frame['likes_number']

In [248]:
train_frame.describe()

,pinned,post_hour,post_month,repost,signed,text_length,with_attachment,000,10,100,...,экономист,экономическии,электричка,юридическии,юрист,юрфак,являться,язык,якобы,ясин
count,1500.0,1500.000000,1500.000000,1500.000000,1500.0,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.0,15.180000,3.732000,0.029333,0.0,239.930667,0.223333,0.000252,0.001951,0.000489,...,0.001051,0.001328,0.000691,0.000534,0.000542,0.000291,0.000997,0.001482,0.000265,0.000243
std,0.0,6.067191,1.664339,0.168795,0.0,357.668800,0.416619,0.004742,0.019470,0.005204,...,0.012615,0.018117,0.011144,0.009194,0.007822,0.004757,0.015235,0.013865,0.005270,0.004234
min,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,12.000000,2.000000,0.000000,0.0,67.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,16.000000,4.000000,0.000000,0.0,134.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,20.000000,5.000000,0.000000,0.0,275.250000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,23.000000,6.000000,1.000000,0.0,5035.000000,1.000000,0.129340,0.441839,0.091979,...,0.334288,0.504354,0.238118,0.213870,0.201969,0.103269,0.462557,0.236528,0.174195,0.097465


Saving train frame to file

In [249]:
# train_frame.to_csv('traindata.csv')

# 4. Comparing different methods

Splitting into train and test samples

In [250]:
from sklearn.cross_validation import train_test_split

# Splitting it into test and train samples
X_train, X_test, y_train, y_test = train_test_split(train_frame, value_frame, test_size=0.3, random_state=42)

Importing libs

In [291]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import numpy as np


In [269]:
def compare(est, param, est_name):
    cv = GridSearchCV(est, param, n_jobs = -1)
    cv.fit(X_train, y_train);
    print('CV best score for', est_name, ': ', cv.best_score_)
    
    predicted = cv.predict(X_test)
    mse = mean_squared_error(y_test, predicted)
    print('MSE for', est_name, ':' , mse)
    r2 = r2_score(y_test, predicted)
    print('R^2 for', est_name, ':' , r2)    

## 4.1 Linear regression

### 4.1.1 Simple linear regression

In [273]:
parameters = {'fit_intercept':[True, False],'normalize':[True, False]}
compare(LinearRegression(), parameters, 'simple linear regression')

CV best score for simple linear regression :  -92.6627023853
MSE for simple linear regression : 389888.390702
R^2 for simple linear regression : -420.516274215


### 4.1.2 Linear regression with L1 regularization

In [274]:
parameters = {'alpha':np.arange(1, 100, 5), 'positive':[True, False],'normalize':[True, False], 
              'selection':['cyclic', 'random']}
compare(Lasso(), parameters, 'linear regression with L1 regularization')

CV best score for linear regression with L1 regularization :  0.224873824274
MSE for linear regression with L1 regularization : 679.171584125
R^2 for linear regression with L1 regularization : 0.26573377787


### 4.1.3 Linear regression with L2 regularization

In [279]:
parameters = {'alpha':np.logspace(1.0, 10.0, 101.00), 'fit_intercept':[True, False],'normalize':[True, False],
              'solver':['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'auto']}
compare(Ridge(), parameters, 'linear regression with L2 regularization')

CV best score for linear regression with L2 regularization :  0.27902482471
MSE for linear regression with L2 regularization : 666.251136282
R^2 for linear regression with L2 regularization : 0.279702336991


## 4.2 Decision trees and random forests

### 4.2.1 DecisionTreeClassifier

In [281]:
parameters = {'presort':[True, False],'max_depth': np.arange(1, 20), 'class_weight':['balanced', None], 'splitter':['random', 'best'],
             'max_features':['auto', 'sqrt', 'log2', None]}
compare(tree.DecisionTreeClassifier(), parameters, 'decision tree classifier')

C:\Users\bulga\Anaconda3\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


CV best score for decision tree classifier :  0.174285714286
MSE for decision tree classifier : 1215.97333333
R^2 for decision tree classifier : -0.314613518214


### 4.2.2 RandomForestClassifier

In [294]:
parameters = {'n_estimators':[10, 20, 30], 'class_weight':['balanced', 'balanced_subsample', None],
             'max_features':['auto', 'sqrt', 'log2', None]}
compare(RandomForestClassifier(), parameters, 'random forest classifier')
# 'max_depth': np.arange(2, 20)

C:\Users\bulga\Anaconda3\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\bulga\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

...........................................................................
C:\Users\bulga\Anaconda3\lib\runpy.py

### 4.3 kNN

In [300]:
parameters = {'leaf_size':np.arange(30, 100, 10),'n_neighbors': np.arange(5, 20), 
              'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
compare(KNeighborsClassifier(), parameters, 'kNN')

C:\Users\bulga\Anaconda3\lib\site-packages\sklearn\cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


CV best score for kNN :  0.150476190476
MSE for kNN : 1027.62666667
R^2 for kNN : -0.11098810364
